In [2]:
import pandas as pd

df =pd.read_csv("../../dataset/romanian_political_articles_v1.csv")

In [3]:
df = df.dropna(subset=["maintext", "source_domain"])

In [ ]:
import re
import spacy
from unidecode import unidecode

nlp = spacy.load("ro_core_news_sm")

def remove_quotes(text):
    return re.sub(r'"[^"]+"', '', text)

def remove_html(text):
    return re.sub(r"<.*?>", "", text)

def remove_digits(text):
    return re.sub(r'\b\d+\b', '', text)

def remove_punctuation(text):
    return re.sub(r'[^\w\s.]', '', text)

def lowercase_first_sentence_word(text):
    doc = nlp(text)
    sentences = []
    for sent in doc.sents:
        words = sent.text.split()
        if words:
            words[0] = words[0].lower()
            sentences.append(' '.join(words))
    return ' '.join(sentences)

def preprocess_text(text : str):
    if not text or len(text.strip()) == 0:
        return ""

    text = remove_quotes(text)

    text = remove_punctuation(text)

    text = remove_digits(text)

    text = remove_html(text)

    text = re.sub(r'\.\s+', ' PERIOD ', text)
    if text.endswith('.'):
        text = text[:-1] + ' PERIOD'

    text = text.strip()

    return lowercase_first_sentence_word(text)

df['cleantext'] = df['maintext'].apply(preprocess_text)

df = df[df['cleantext'].str.split().str.len() > 100]

df.head()

,url,title,date_publish,description,maintext,source_domain,authors,cleantext
1,https://www.realitatea.net/stiri/politica/scan...,Scandalurile prin care a devenit celebru candi...,2025-04-13 13:09:59,Tupeu incredibil din partea lui Nicusor Dan! S...,Activistă și ea din zona soroșistă ce a venit ...,www.realitatea.net,Georgiana Balaban,activistă și ea din zona soroșistă ce a venit ...
2,https://www.realitatea.net/stiri/politica/crin...,"Crin Antonescu: „Nicușor dă prea puțină apă, P...",2025-04-12 20:54:54,",,Nicusor Dan da prea putina apa si Ponta prea...","Crin Antonescu: „Ei promit lapte și miere, dar...",www.realitatea.net,Georgiana Balaban,crin Antonescu Ei promit lapte și miere dar nu...
3,https://www.realitatea.net/stiri/politica/lasc...,Lasconi: „Nicușor Dan mi-a cerut să mă retrag ...,2025-04-12 19:02:32,",,Nicusor Dan mi-a cerut sa ma duc la Stejarii...",Candidata la prezidențiale trădată de propriul...,www.realitatea.net,Georgiana Balaban,candidata la prezidențiale trădată de propriul...
5,https://www.realitatea.net/stiri/politica/geor...,George Simion: „Ideea de tur doi înapoi înseam...,2025-04-12 21:50:26,",,Ideea de tur doi inapoi inseamna revenirea l...",Totul pentru a evita o situație asemănătoare c...,www.realitatea.net,Georgiana Balaban,totul pentru a evita o situație asemănătoare c...
6,https://www.realitatea.net/stiri/politica/flor...,Florin Zamfirescu: „Oamenii nu mai au cu cine ...,2025-04-13 08:54:16,Florin Zamfirescu a vorbit in exclusivitate la...,Actorul spune că românii sunt îngenuncheați și...,www.realitatea.net,Georgiana Balaban,actorul spune că românii sunt îngenuncheați și...


In [5]:
def extract_ngrams_spacy(text, n):
    doc = nlp(text)
    tokens = [
        token.text for token in doc
        if not token.is_space and not token.is_punct and token.is_alpha
    ]

    return [' '.join(tokens[i:i+n]) for i in range(len(tokens) - n + 1)]


df['monograms'] = df['cleantext'].apply(lambda t: extract_ngrams_spacy(t, 1))
df['bigrams'] = df['cleantext'].apply(lambda t: extract_ngrams_spacy(t, 2))
df['trigrams'] = df['cleantext'].apply(lambda t: extract_ngrams_spacy(t, 3))
df['all_ngrams'] = df['bigrams'] + df['trigrams'] + df['monograms']
df.head()

,url,title,date_publish,description,maintext,source_domain,authors,cleantext,monograms,bigrams,trigrams,all_ngrams
1,https://www.realitatea.net/stiri/politica/scan...,Scandalurile prin care a devenit celebru candi...,2025-04-13 13:09:59,Tupeu incredibil din partea lui Nicusor Dan! S...,Activistă și ea din zona soroșistă ce a venit ...,www.realitatea.net,Georgiana Balaban,activistă și ea din zona soroșistă ce a venit ...,"[activistă, și, ea, din, zona, soroșistă, ce, ...","[activistă și, și ea, ea din, din zona, zona s...","[activistă și ea, și ea din, ea din zona, din ...","[activistă și, și ea, ea din, din zona, zona s..."
2,https://www.realitatea.net/stiri/politica/crin...,"Crin Antonescu: „Nicușor dă prea puțină apă, P...",2025-04-12 20:54:54,",,Nicusor Dan da prea putina apa si Ponta prea...","Crin Antonescu: „Ei promit lapte și miere, dar...",www.realitatea.net,Georgiana Balaban,crin Antonescu Ei promit lapte și miere dar nu...,"[crin, Antonescu, Ei, promit, lapte, și, miere...","[crin Antonescu, Antonescu Ei, Ei promit, prom...","[crin Antonescu Ei, Antonescu Ei promit, Ei pr...","[crin Antonescu, Antonescu Ei, Ei promit, prom..."
3,https://www.realitatea.net/stiri/politica/lasc...,Lasconi: „Nicușor Dan mi-a cerut să mă retrag ...,2025-04-12 19:02:32,",,Nicusor Dan mi-a cerut sa ma duc la Stejarii...",Candidata la prezidențiale trădată de propriul...,www.realitatea.net,Georgiana Balaban,candidata la prezidențiale trădată de propriul...,"[candidata, la, prezidențiale, trădată, de, pr...","[candidata la, la prezidențiale, prezidențiale...","[candidata la prezidențiale, la prezidențiale ...","[candidata la, la prezidențiale, prezidențiale..."
5,https://www.realitatea.net/stiri/politica/geor...,George Simion: „Ideea de tur doi înapoi înseam...,2025-04-12 21:50:26,",,Ideea de tur doi inapoi inseamna revenirea l...",Totul pentru a evita o situație asemănătoare c...,www.realitatea.net,Georgiana Balaban,totul pentru a evita o situație asemănătoare c...,"[totul, pentru, a, evita, o, situație, asemănă...","[totul pentru, pentru a, a evita, evita o, o s...","[totul pentru a, pentru a evita, a evita o, ev...","[totul pentru, pentru a, a evita, evita o, o s..."
6,https://www.realitatea.net/stiri/politica/flor...,Florin Zamfirescu: „Oamenii nu mai au cu cine ...,2025-04-13 08:54:16,Florin Zamfirescu a vorbit in exclusivitate la...,Actorul spune că românii sunt îngenuncheați și...,www.realitatea.net,Georgiana Balaban,actorul spune că românii sunt îngenuncheați și...,"[actorul, spune, că, românii, sunt, îngenunche...","[actorul spune, spune că, că românii, românii ...","[actorul spune că, spune că românii, că români...","[actorul spune, spune că, că românii, românii ..."


In [6]:
from collections import Counter
import numpy as np

all_phrases = list(set(phrase for phrases_list in df['all_ngrams'] for phrase in phrases_list))

# Create the Nij matrix (rows: phrases, columns: domains)
domains = df['source_domain'].unique()
nij_matrix = pd.DataFrame(0, index=all_phrases, columns=domains)

# Fill the Nij matrix
for domain in domains:
    domain_articles = df[df['source_domain'] == domain]
    domain_phrases = Counter()

    for phrases_list in domain_articles['all_ngrams']:
        domain_phrases.update(phrases_list)

    for phrase, count in domain_phrases.items():
        nij_matrix.at[phrase, domain] = count

In [7]:
print("The frequency matrix has: {} elements", len(nij_matrix.index))

The frequency matrix has: {} elements 1757767


In [8]:
from collections import defaultdict

monograms = set(mono for monograms_list in df['monograms'] for mono in monograms_list)
bigrams = set(bi for bigrams_list in df['bigrams'] for bi in bigrams_list)
trigrams = set(tri for trigrams_list in df['trigrams'] for tri in trigrams_list)

def build_reverse_index_by_ngram(longer_phrases, ngram_size):
    """
    Builds a reverse index where keys are subphrases (of size `ngram_size`)
    and values are lists of longer phrases that contain them.
    """
    reverse_index = defaultdict(list)

    for phrase in longer_phrases:
        tokens = phrase.split()
        for i in range(len(tokens) - ngram_size + 1):
            subphrase = ' '.join(tokens[i:i+ngram_size])
            reverse_index[subphrase].append(phrase)

    return reverse_index

def find_phrases_to_delete_fast(shorter_phrases, reverse_index, nij_matrix, threshold=0.7):
    """
    Fast version: avoid nested loops by reverse-indexing longer phrases.
    Assumes all phrases are strings.
    """
    phrases_to_delete = []

    for short in shorter_phrases:
        short_count = nij_matrix.loc[short].sum()

        candidates = reverse_index.get(short)

        if not candidates:
            continue

        for long_phrase in candidates:
            long_count = nij_matrix.loc[long_phrase].sum()
            if long_count / short_count >= threshold:
                phrases_to_delete.append(short)
                break

    return phrases_to_delete

reverse_index_bigrams = build_reverse_index_by_ngram(bigrams, 1)
reverse_index_trigrams = build_reverse_index_by_ngram(trigrams, 1)
reverse_index_bigrams_in_trigrams = build_reverse_index_by_ngram(trigrams, 2)


monograms_in_bigrams_to_delete = find_phrases_to_delete_fast(monograms, reverse_index_bigrams, nij_matrix, threshold=0.7)
monograms_in_trigrams_to_delete =  find_phrases_to_delete_fast(monograms, reverse_index_trigrams, nij_matrix, threshold=0.7)
bigrams_to_delete = find_phrases_to_delete_fast(bigrams, reverse_index_bigrams_in_trigrams, nij_matrix, threshold=0.7)

nij_matrix = nij_matrix.drop(index=
                             monograms_in_bigrams_to_delete +
                             monograms_in_trigrams_to_delete +
                             bigrams_to_delete,
                             errors='ignore')

In [9]:
print("The frequency matrix has: {} elements", len(nij_matrix.index))

The frequency matrix has: {} elements 1269994


In [10]:
phrase_totals = nij_matrix.sum(axis=1)

phrase_max_per_domain = nij_matrix.max(axis=1)

dominance_ratio = phrase_max_per_domain / phrase_totals

too_good_phrases = dominance_ratio[dominance_ratio > 0.9].index

nij_matrix = nij_matrix.drop(index=too_good_phrases, errors='ignore')

In [11]:
print("The frequency matrix has: {} elements", len(nij_matrix.index))

The frequency matrix has: {} elements 261680


In [12]:
nij_matrix = nij_matrix[~nij_matrix.index.str.contains("PERIOD")]

In [13]:
print("The frequency matrix has: {} elements", len(nij_matrix.index))

The frequency matrix has: {} elements 235814


In [14]:
n_total = nij_matrix.values.sum()

# Joint probability matrix: P_ij
p_ij = nij_matrix / n_total

p_i = p_ij.sum(axis=1)
p_j = p_ij.sum(axis=0)

info_scores = []

for phrase in p_ij.index:
    score = 0
    for domain in p_ij.columns:
        pij = p_ij.at[phrase, domain]
        if pij > 0:
            pi = p_i[phrase]
            pj = p_j[domain]
            score += pij * np.log2(pij / (pi * pj))
    info_scores.append(score)

info_scores = pd.Series(info_scores, index=p_ij.index).sort_values(ascending=False)

In [16]:
top_5000 = info_scores.head(5000)
with open("../../dataset/top_5000_info_scores.txt", "w", encoding="utf-8") as f:
    f.write(top_5000.to_string())
